In [ ]:
from google.colab import drive
import numpy as np
import scipy.io
import time
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + ''
import os
import sys
sys.path.append(base_dir)
print(os.getcwd())
print(os.listdir())
os.chdir(base_dir)
print(os.getcwd())
print(os.listdir())

In [ ]:
!pip install pypower
import pypower
from pypower.case14 import case14
from pypower.case9 import case9
from pypower.case118 import case118
from pypower.case300 import case300

from pypower.case9 import case9
from pypower.case4gs import case4gs
from pypower.case30 import case30

from pypower.opf import opf
from pypower.dcopf import dcopf
from pypower.makeYbus import makeYbus

import numpy as np
import random
from scipy.optimize import minimize, fsolve
import torch
from scipy.optimize import Bounds
import pandas as pd

class environment():
    def __init__(self, network):

        #make bus numbers 0-N
        df = pd.DataFrame([network['bus'][:,0]]).T
        v = df.stack().unique()
        v.sort()
        f = pd.factorize(v)
        m = pd.Series(f[0], f[1])
        df = df.stack().map(m).unstack()
        network['bus'][:,0] = df.to_numpy().reshape((len(df.to_numpy()),))
        df = pd.DataFrame([network['gen'][:,0]]).T
        df = df.stack().map(m).unstack()
        network['gen'][:,0] = df.to_numpy().reshape((len(df.to_numpy()),))
        df = pd.DataFrame([network['branch'][:,0]]).T
        df = df.stack().map(m).unstack()
        network['branch'][:,0] = df.to_numpy().reshape((len(df.to_numpy()),))
        df = pd.DataFrame([network['branch'][:,1]]).T
        df = df.stack().map(m).unstack()
        network['branch'][:,1] = df.to_numpy().reshape((len(df.to_numpy()),))

        if network['bus'][0][0]==1:
            network['bus'][:,0] = network['bus'][:,0] - 1
            network['gen'][:,0] = network['gen'][:,0] - 1
            network['branch'][:,[0,1]] = network['branch'][:,[0,1]] - [1, 1]

        self.network = network
        self.num_buses = len(network['bus'])
        self.num_gens = len(network['gen'])
        self.num_branches = len(network['branch'])
        self.t = 0
        self.state = []
        self.baseMVA = network['baseMVA']
        self.balance = 0
        self.limits = self.get_limits()
        self.edge_index = torch.tensor(np.array([self.network['branch'][:, 0], self.network['branch'][:, 1]]), dtype=torch.int64)
        self.ybus, _, _ = makeYbus(self.baseMVA, self.network['bus'], self.network['branch'])
        self.max_load = np.sum(self.network['gen'][:,8]), np.sum(self.network['gen'][:,3])

    #reset environment
    def reset(self):
        self.t = 0
        self.update_state()
        return self.state

    #make a test set
    def make_test_data(self):
        X = []
        Y = []
        self.update_state()
        for i in range(5001):
            y = []
            X.append(np.array(self.state).flatten())
            print(np.shape(X))
            self.change_load()
            self.network = opf(self.network)
            theta = self.network['bus'][:,8]
            v = self.network['bus'][:,7]
            P_G = self.network['gen'][:,1]/self.baseMVA
            Q_G = self.network['gen'][:,2]/self.baseMVA
            self.update_state()
            y.extend(theta)
            y.extend(v)
            y.extend(P_G)
            y.extend(Q_G)
            Y.append(y)
            if i % 5000 == 0:
                np.savetxt('/content/gdrive/MyDrive/MTS Datasets/Regression/OPF/' + "ACOPF_Case118Data_X_TEST" + str(i) + ".csv", np.array(X), delimiter =",")
                np.savetxt('/content/gdrive/MyDrive/MTS Datasets/Regression/OPF/' + "ACOPF_Case118Data_y_TEST" + str(i) + ".csv", Y, delimiter =",")


    # get limits for inequality constraints
    def get_limits(self):
        lower = []
        upper = []

        for bus in self.network['bus']:
            lower.append(-360)
            upper.append(360)
        for bus in self.network['bus']:
            lower.append(bus[12])
            upper.append(bus[11])
        for gen in self.network['gen']:
            lower.append(gen[9]/self.baseMVA)
            upper.append(gen[8]/self.baseMVA)
        for gen in self.network['gen']:
            lower.append(gen[4]/self.baseMVA)
            upper.append(gen[3]/self.baseMVA)
        return lower, upper


    #step from s_t to s_t+1,
    def step(self, actions):
        self.t += 1
        self.change_load()
        self.update_state()
        self.ybus, _, _ = makeYbus(self.baseMVA, self.network['bus'], self.network['branch'])
        return self.state, 0


    def update_state(self):
        x = []
        for j in range(self.num_buses):
            x.append([self.network['bus'][j][2], self.network['bus'][j][3]])
        self.state = x


    def change_load(self):
        load = np.random.randint(self.max_load[0] * 0.1, self.max_load[0] * 0.85)
        loads = (np.random.dirichlet(np.ones(self.num_buses)*4, size=1) * load)[0]
        for i in range(self.num_buses):
            if self.network['bus'][i][2] == 0:
                continue
            split = (random.randint(1, 2000) / 10000)
            self.network['bus'][i][2] = loads[i] * (1-split)
            self.network['bus'][i][3] = loads[i] * (split)

    #in pypower.pipsopf_solver, add the following code after line 113:
    #x0 = ppc['x0']
    def run_opf(self, x0):
        self.network['x0'] = x0
        self.network = opf(self.network)


In [ ]:
!pip install torch_geometric
import os
import time
import torch.nn as nn
from torch.distributions import MultivariateNormal
from torch.distributions import Categorical
from torch_geometric.nn import GCNConv

device = torch.device('cpu')

if(torch.cuda.is_available()):
    device = torch.device('cuda:0')
    torch.cuda.empty_cache()



class Actor(torch.nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        self.conv1 = GCNConv(state_dim[0], 64)
        self.conv2 = GCNConv(64, 128)
        self.conv3 = GCNConv(128, 64)
        self.out = nn.Linear(state_dim[1] * 64, action_dim)

    def forward(self, x, edge_index):
        if len(x.shape) == 2:
            x = x.unsqueeze(0)
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()
        out = self.out(torch.flatten(h, start_dim=1)).sigmoid()
        return out

# class Actor(torch.nn.Module):
#     def __init__(self, state_dim, action_dim):
#         super(Actor, self).__init__()
#         self.linear1 = nn.Linear(state_dim[0] * state_dim[1], 64)
#         self.linear2 = nn.Linear(64, 128)
#         self.linear3 = nn.Linear(128, 64)
#         self.out = nn.Linear(64, action_dim)

#     def forward(self, x, edge_index):
#         if len(x.shape) == 2:
#             x = x.unsqueeze(0)
#         h = self.linear1(x)
#         h = h.tanh()
#         h = self.linear2(h)
#         h = h.tanh()
#         h = self.linear3(h)
#         h = h.tanh()
#         out = self.out(h).sigmoid()
#         return out


class Critic(torch.nn.Module):
    def __init__(self, state_dim):
        super(Critic, self).__init__()
        self.conv1 = GCNConv(state_dim[0], 64)
        self.conv2 = GCNConv(64, 128)
        self.conv3 = GCNConv(128, 64)
        self.out = nn.Linear(state_dim[1] * 64, 1)

    def forward(self, x, edge_index):
        if len(x.shape) == 2:
            x = x.unsqueeze(0)
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()
        out = self.out(torch.flatten(h, start_dim=1))
        return out

# class Critic(torch.nn.Module):
#     def __init__(self, state_dim):
#         super(Critic, self).__init__()
#         self.linear1 = nn.Linear(state_dim[0] * state_dim[1], 64)
#         self.linear2 = nn.Linear(64, 128)
#         self.linear3 = nn.Linear(128, 64)
#         self.out = nn.Linear(64, 1)

#     def forward(self, x, edge_index):
#         if len(x.shape) == 2:
#             x = x.unsqueeze(0)
#         h = self.linear1(x)
#         h = h.tanh()
#         h = self.linear2(h)
#         h = h.tanh()
#         h = self.linear3(h)
#         h = h.tanh()
#         out = self.out(h)
#         return out


class Buffer:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
        self.state_values = []
        self.is_terminals = []


    def clear(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]
        del self.state_values[:]
        del self.is_terminals[:]


class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, edge_index, action_sd_init):
        super(ActorCritic, self).__init__()

        self.edge_index = edge_index.to(device)
        self.action_dim = action_dim
        self.action_var = torch.full((action_dim,), action_sd_init * action_sd_init).to(device)

        self.actor = Actor(state_dim, action_dim)

        self.critic = Critic(state_dim)

    def set_action_sd(self, new_action_sd):
        self.action_var = torch.full((self.action_dim,), new_action_sd * new_action_sd).to(device)

    def forward(self):
        raise NotImplementedError


    def act(self, state):
        action_mean = self.actor(state, self.edge_index)
        cov_mat = torch.diag(self.action_var).unsqueeze(dim=0).float()
        dist = MultivariateNormal(action_mean, cov_mat)

        action = dist.sample()
        action_logprob = dist.log_prob(action)
        state_val = self.critic(state, self.edge_index)

        return action.detach(), action_logprob.detach(), state_val.detach()


    def evaluate(self, state, action):
        action_mean = self.actor(state, self.edge_index)
        action_var = self.action_var.expand_as(action_mean)
        cov_mat = torch.diag_embed(action_var).to(device)
        dist = MultivariateNormal(action_mean, cov_mat)

        if self.action_dim == 1:
            action = action.reshape(-1, self.action_dim)


        action_logprobs = dist.log_prob(action)
        S = dist.entropy()
        state_values = self.critic(state, self.edge_index)

        return action_logprobs, state_values, S


class PPO:
    def __init__(self, state_dim, action_dim, lr_actor, lr_critic, gamma, epochs, eps_clip, edge_index, action_sd):

        self.action_sd = action_sd

        self.gamma = gamma
        self.eps_clip = eps_clip
        self.epochs = epochs

        self.buffer = Buffer()

        self.policy = ActorCritic(state_dim, action_dim, edge_index, action_sd).to(device)
        self.optimizer = torch.optim.Adam([
                        {'params': self.policy.actor.parameters(), 'lr': lr_actor},
                        {'params': self.policy.critic.parameters(), 'lr': lr_critic}
                    ])

        self.policy_old = ActorCritic(state_dim, action_dim, edge_index, action_sd).to(device)
        self.policy_old.load_state_dict(self.policy.state_dict())

        self.MseLoss = nn.MSELoss()


    def set_action_sd(self, new_action_sd):

        self.action_sd = new_action_sd
        self.policy.set_action_sd(new_action_sd)
        self.policy_old.set_action_sd(new_action_sd)


    def decay_action_sd(self, action_sd_decay_rate, min_action_sd):
        self.action_sd = self.action_sd - action_sd_decay_rate
        self.action_sd = round(self.action_sd, 4)
        if (self.action_sd <= min_action_sd):
            self.action_sd = min_action_sd
            print("action_sd : ", self.action_sd)
        else:
            print("action_sd : ", self.action_sd)
        self.set_action_sd(self.action_sd)

    def select_action(self, state):
        with torch.no_grad():
            state = torch.FloatTensor(state).to(device)
            action, action_logprob, state_val = self.policy_old.act(state)

        self.buffer.states.append(state)
        self.buffer.actions.append(action)
        self.buffer.logprobs.append(action_logprob)
        self.buffer.state_values.append(state_val)

        return action.detach().cpu().numpy().flatten()



    def update(self):

        # estimate of returns
        rewards = []
        discounted_reward = 0
        for reward, is_terminal in zip(reversed(self.buffer.rewards), reversed(self.buffer.is_terminals)):
            if is_terminal:
                discounted_reward = 0
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        rewards = torch.tensor(rewards, dtype=torch.float32).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-7)

        #  to tensor
        old_states = torch.squeeze(torch.stack(self.buffer.states, dim=0)).detach().to(device)
        old_actions = torch.squeeze(torch.stack(self.buffer.actions, dim=0)).detach().to(device)
        old_logprobs = torch.squeeze(torch.stack(self.buffer.logprobs, dim=0)).detach().to(device)
        old_state_values = torch.squeeze(torch.stack(self.buffer.state_values, dim=0)).detach().to(device)

        # calculate advantages
        advantages = rewards.detach() - old_state_values.detach()


        # Optimize policy
        for _ in range(self.epochs):

            # Evaluate old actions and values
            logprobs, state_values, S = self.policy.evaluate(old_states, old_actions)

            # match state_values tensor dimensions with rewards tensor
            state_values = torch.squeeze(state_values)

            # ratio (pi_theta / pi_theta__old)
            ratios = torch.exp(logprobs - old_logprobs.detach())

            surrogate1 = ratios * advantages
            surrogate2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages

            # objective function
            loss = -torch.min(surrogate1, surrogate2) + 0.5 * self.MseLoss(state_values, rewards) - 0.01 * S


            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()

        self.policy_old.load_state_dict(self.policy.state_dict())

        self.buffer.clear()


    def save(self, checkpoint_path):
        torch.save(self.policy_old.state_dict(), checkpoint_path)


    def load(self, checkpoint_path):
        self.policy_old.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))
        self.policy.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))





In [ ]:
# initialize hyperparameters

env_name = "Case 118"

episode_len = 1000
max_training_timesteps = episode_len*100

print_freq = episode_len
log_freq = episode_len
save_model_freq = 250

action_sd = 0.25     #action standard deviation
action_sd_decay_rate = 0.01
min_action_sd = 0.0000000001
action_sd_decay_freq = 250

T = episode_len * 1     # update policy every n timesteps
epochs = 100               # update policy for K epochs
eps_clip = 0.1             # clip parameter for PPO
gamma = 0.001              # discount factor
lr_actor = 0.000005      # learning rate for actor network
lr_critic = 0.000005      # learning rate for critic network

random_seed = 0         # set random seed if required (0 = no random seed)


env = environment(case118())
# env = environment(case14())

limits = env.get_limits()

# state space dimension
state_dim = [2, env.num_buses]
# action space dimension
action_dim = 2 * env.num_buses + 2 * env.num_gens


directory = "PPO_preTrained"
if not os.path.exists(directory):
      os.makedirs(directory)

directory = directory + '/' + env_name + '/'
if not os.path.exists(directory):
      os.makedirs(directory)


checkpoint_path = directory + "PPO_{}.pth".format(env_name)

if random_seed:
    torch.manual_seed(random_seed)
    env.seed(random_seed)
    np.random.seed(random_seed)


# initialize agent
ppo_agent = PPO(state_dim, action_dim, lr_actor, lr_critic, gamma, epochs, eps_clip, env.edge_index, action_sd)
# ppo_agent.load()



print_running_reward = 0
print_running_episodes = 0


t = 0
i_episode = 0

log_data = []

# training loop

while t <= max_training_timesteps:

    state = env.reset()
    current_ep_reward = 0
    diverge_counter = 0
    converge_times = 0

    for t in range(0, episode_len):
        action = ppo_agent.select_action(state)
        for i in range(len(action)):
            ll = limits[0][i]
            uu = limits[1][i]
            action[i] = action[i] * (uu - ll) + ll

        reward = time.time()
        env.run_opf(action)

        # rewards
        reward = -(time.time() - reward)
        if env.network['success'] == False:
            diverge_counter = diverge_counter + 1
            reward = -5
        else:
            converge_times = converge_times - reward

        state, done = env.step(action)
        # saving reward
        ppo_agent.buffer.rewards.append(reward)
        ppo_agent.buffer.is_terminals.append(done)

        t +=1
        current_ep_reward += reward

        # update agent
        if t % T == 0:
            ppo_agent.update()

        # decay action sd of ouput action distribution
        if t % action_sd_decay_freq == 0:
            ppo_agent.decay_action_sd(action_sd_decay_rate, min_action_sd)


        # printing average reward
        if t % print_freq == 0:
            print(converge_times/(print_freq-diverge_counter), diverge_counter)
            log_data.append([diverge_counter, converge_times/(print_freq-diverge_counter), current_ep_reward / print_freq])

            print_avg_reward = current_ep_reward / print_freq
            print_avg_reward = round(print_avg_reward, 2)

            print("Episode : {} \t\t Timestep : {} \t\t Average Reward : {}".format(i_episode, t, print_avg_reward))

            np.savetxt(checkpoint_path + "log_data", log_data, delimiter =", ", fmt ='% s')

            print_running_reward = 0
            print_running_episodes = 0
            diverge_counter = 0
            converge_times = 0

        # save model
        if t % save_model_freq == 0:
            print("saving model at : " + checkpoint_path)
            ppo_agent.save(checkpoint_path)

    print_running_reward += current_ep_reward
    print_running_episodes += 1


    i_episode += 1


In [ ]:
env = environment(case118())
X = np.loadtxt('ACOPF_Case14Data_X_TEST5000.csv', delimiter=",", dtype=None)
divergences = 0
r_total = 0
div_count = 0
for i in range(len(X)):
    x = X[i]
    state = []
    for j in range(int(len(x)/2)):
        state.append([x[j*2], x[j*2+1]])
        env.network['bus'][j][2] = x[j*2]
        env.network['bus'][j][3] = x[j*2+1]
    action = ppo_agent.select_action(state)
    for k in range(len(action)):
        ll = limits[0][k]
        uu = limits[1][k]
        action[k] = action[k] * (uu - ll) + ll
    r = time.time()
    env.run_opf(action)
    r = time.time() - r
    if env.network['success'] == False:
        div_count+=1
    r_total += r